# Reto: Modelo basado en *deep learning* para la discriminación entre gatos y perros

**Objetivos**:
- Diseñar modelos de redes neuronales profundas (*deep learning*), enfocándose en la clasificación de imágenes de acuerdo a categrías preestablecidas para resolver problemas con relevancia social, permitiendo valor en los diversos sectores.
- Crear modelos de datos *Deep Neural Networks* (DNN) utilizando PyTorch basado en Python; seleccionando el modelo adecuado y analizando la exactitud del modelo, para cumplir lo mejor posible con los requerimientos de la tarea requerida.

## 1. Preprocesamiento de la base de datos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Importando bibliotecas
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torchsummary import summary
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pathlib

In [7]:
# Eliminando cualquier archivo que no sea JPG, JPEG, PNG
!find /content/drive/MyDrive/Reto_PyTorch/catsvsdogs -type f ! -name '*.jpg' ! -name '*.jpeg' ! -name '.*png' -delete

In [8]:
# Directorio base
data_dir = pathlib.Path('/content/drive/MyDrive/Reto_PyTorch/catsvsdogs')
conteo_img = len(list(data_dir.glob('*/*')))
print('Total de imágenes a analizar:', conteo_img)

Total de imágenes a analizar: 4895


In [11]:
# Transformaciones
transform = transforms.Compose([
    # Ajustar tamaño de la imagen
    transforms.Resize((224,224)),
    # Transformar a tensor
    transforms.ToTensor(),
])

### 1.2 Normalización de las imágenes

In [18]:
# Imagenes sin normalizar
dataset = datasets.ImageFolder(data_dir, transform=transform)
loader = DataLoader(dataset,batch_size=64,shuffle=False,num_workers=2)

# Calculando promedio y desviación estándar
mean = 0
std = 0
total_imgs = 0
for images,_ in loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples,images.size(1),-1) # (batch,canales,pixeles)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    total_imgs += batch_samples
mean = mean/total_imgs
std = std/total_imgs
print('Promedio:', mean)
print('Desviación estándar:', std)

Promedio: tensor([0.4815, 0.4475, 0.3969])
Desviación estándar: tensor([0.2255, 0.2230, 0.2250])


In [20]:
# Transformación final
transform = transforms.Compose([
    # Ajustar tamaño de la imagen
    transforms.Resize((224,224)),
    # Transformar a tensor
    transforms.ToTensor(),
    # Normalizar
    transforms.Normalize(mean = [0.4815,0.4475,0.3969],
                        std = [0.2255,0.2230,0.2250])
])


## 2. Sets de Entrenamiento y Validación

In [21]:
# Dataset completo desde el directorio establecido
dataset = datasets.ImageFolder(data_dir, transform=transform)

# División: 80% entrenamiento, 20% validación
train_size = int(0.8*len(dataset))
val_size = len(dataset) - train_size

train_set, val_set = random_split(dataset, [train_size, val_size])

# Batch
batch_size = 32
train_loader = DataLoader(train_set,batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_set,batch_size=batch_size,shuffle=False)

# Tamaño de los sets
print(f'Imágenes:',len(dataset))
print(f'Entrenamiento', len(train_set))
print(f'Validación', len(val_set))

Imágenes: 4895
Entrenamiento 3916
Validación 979


## 3. Construcción de la Red Neuronal

In [29]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self,input_size = 224*224*3,output_size = 2):
        super().__init__()
        self.fc1 = nn.Linear(input_size,512)
        self.fc2 = nn.Linear(512,128)
        self.fc3 = nn.Linear(128,64)
        self.fc4 = nn.Linear(64,output_size)

    def forward(self,X):
        X = X.view(X.shape[0],-1)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = F.relu(self.fc3(X))
        X = self.fc4(X)
        return X

 # Creando el modelo
model = MultiLayerPerceptron()

# Ver la estructura del modelo
print(model)

MultiLayerPerceptron(
  (fc1): Linear(in_features=150528, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=2, bias=True)
)


In [31]:
# Definiendo pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

# Mover a GPU si está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

MultiLayerPerceptron(
  (fc1): Linear(in_features=150528, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=2, bias=True)
)

## 4. Entrenamiento de la Red Neuronal